Here are the imports

In [6]:
import folium
import pandas as pd
import numpy as np
import os
import geopandas as gpd

(somewhat inefficiently) narrowing down the set to be only the zipcodes in the Building_Energy_Benchmarking file

In [ ]:

#this is the file that makes the map
county_map = gpd.read_file('Zip_Codes.geojson')

#I know this is so redundant but I couldn't figure out how to combine them
county_map = county_map[county_map["ZIPCODE"] > "98100"]
county_map = county_map[county_map["ZIPCODE"] < "98200"]

county_map = county_map[county_map["ZIPCODE"] != "98110"]
county_map = county_map[county_map["ZIPCODE"] != "98168"]
county_map = county_map[county_map["ZIPCODE"] != "98166"]
county_map = county_map[county_map["ZIPCODE"] != "98148"]
county_map = county_map[county_map["ZIPCODE"] != "98158"]
county_map = county_map[county_map["ZIPCODE"] != "98188"]
#renaming it seattle_map for the last one, since now its only zipcodes in seattle
seattle_map = county_map[county_map["ZIPCODE"] != "98198"]



Building_Energy_Benchmarking has stats on a bunch of different energy use benchmarks for almost all big-ish (>20,000 sqft) buildings in the city. AND the zipcode and coordinates of those buildings, but we are just using the zipcodes for this one. 
The statistic I'm using for this map is the SiteEUI(kBtu/sf), which they describe as:
Site Energy Use Intensity (EUI) is a property's Site Energy Use divided by its gross floor area. Site Energy Use is the annual amount of all the energy consumed by the property on-site, as reported on utility bills. Site EUI is measured in thousands of British thermal units (kBtu) per square foot.

In [9]:
#the energy use file
energy_use = pd.read_csv('2020_Building_Energy_Benchmarking.csv')

#trimming the file down to two building stats, ZipCode and SiteEUI
energy_use = energy_use[['ZipCode', 'SiteEUI(kBtu/sf)']]

cleaning the data, turning it into an integer (to get rid of the .0), and turning it into a string, since the zipcodes in seattle_map are strings.

In [12]:
energy_use = energy_use[energy_use['ZipCode'].notna()]
energy_use['ZipCode'] = energy_use['ZipCode'].astype(np.int64)
energy_use['ZipCode'] = energy_use['ZipCode'].astype(str)
#what that table looks like
energy_use

,ZipCode,SiteEUI(kBtu/sf)
0,98101,54.6
1,98101,40.7
2,98101,44.1
3,98101,48.6
4,98121,67.8
...,...,...
3623,98103,NaN
3624,98109,27.5
3625,98119,104.1
3626,98122,47.9


grouping the SiteEUI data by zipcode and averaging each zipcode's data. 

In [11]:
av_energy_use = energy_use.groupby(["ZipCode"], as_index=False).mean()
#what that table looks like now
av_energy_use

,ZipCode,SiteEUI(kBtu/sf)
0,98101.0,60.731197
1,98102.0,43.325281
2,98103.0,40.795122
3,98104.0,50.024528
4,98105.0,59.957767
5,98106.0,35.181481
6,98107.0,47.558621
7,98108.0,42.134921
8,98109.0,53.726837
9,98112.0,57.123288


In [ ]:

energy_use_per_zip_map = folium.Map(
    location = [47.608013, -122.335167], 
    tiles = 'https://{s}.tile.openstreetmap.de/{z}/{x}/{y}.png',
    attr = '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',
    width = '100%',
    height = '100%',
    min_zoom = 11,
    zoom_start = 11 
    )



In [ ]:


folium.Choropleth(
    geo_data = seattle_map,
    name = 'choropleth',
    data = av_energy_use,
    columns = ['ZipCode','SiteEUI(kBtu/sf)'],
    key_on = 'feature.properties.ZIPCODE',
    bins = 6,
    fill_color = 'OrRd',
    fill_opacity = .7,
    legend_name = 'Site Energy Use Intensity (kBtu/sqft)',
    overlay = False
).add_to(energy_use_per_zip_map)


In [ ]:
folium.GeoJson(
    data = seattle_map,
    tooltip = folium.GeoJsonTooltip(fields = ['ZIPCODE']),
    zoom_on_click = True
).add_to(energy_use_per_zip_map)

In [ ]:
energy_use_per_zip_map